# Environment

In [67]:
!pip3 install mlcroissant
!pip3 install rdflib
!pip3 install pandas
import mlcroissant as mlc
from rdflib import Graph, Namespace, URIRef, Literal
import json
import pandas as pd

# Load HuggingFace RDF

In [68]:
# Smaller dataset
# g = Graph().parse('https://github.com/david4096/huggingface-rdf/releases/download/data-release/huggingface.ttl?download=true', format='ttl')
# Larger dataset
g = Graph().parse('https://huggingface.co/datasets/david4096/huggingface-ttl/resolve/main/huggingface-30k.ttl?download=true', format='ttl')


# Queries

Query to get property counts by dataset

List of classes:

*   https://schema.org/Dataset
    *   http://mlcommons.org/croissant/FileObject / http://mlcommons.org/croissant/FileSet
        *   http://mlcommons.org/croissant/RecordSet
            *   http://mlcommons.org/croissant/Field

The following can not be obtained currently because they are described with blank nodes:
*   http://mlcommons.org/croissant/DataSource
    *   http://mlcommons.org/croissant/Extract
    *   http://mlcommons.org/croissant/Transform
    *   http://mlcommons.org/croissant/Format



In [69]:
# @title Query number of each property of each class
class_URI = "https://schema.org/Dataset" # @param {"type":"string","placeholder":"https://schema.org/Dataset"}
query = """


SELECT ?dataset ?dataset_name ?predicate (count(?o) as ?predicate_count)
WHERE {
    ?dataset <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <"""+class_URI+"""> .
    ?dataset ?predicate ?o .
    OPTIONAL { ?dataset <https://schema.org/name> ?dataset_name . }
}
GROUP BY ?predicate ?dataset
"""
print(query)

qres = g.query(query)
# for row in qres:
#     print(f"Dataset: {row.dataset_name} Predicate: {row.predicate} Count: {row.predicate_count}")




SELECT ?dataset ?dataset_name ?predicate (count(?o) as ?predicate_count)
WHERE {
    ?dataset <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://schema.org/Dataset> .
    ?dataset ?predicate ?o .
    OPTIONAL { ?dataset <https://schema.org/name> ?dataset_name . }
}
GROUP BY ?predicate ?dataset



# Export data

Export to TSV

In [70]:
with open(class_URI.split("/")[-1]+"_query_results.tsv", "w") as outfile:
  first_row = True
  for row in qres:
    if first_row:
      for label in row.labels.keys():
        outfile.write(label + "\t")
      outfile.write("\n")
      first_row = False
    for value in row:
      outfile.write(value + "\t")
    outfile.write("\n")

In [71]:
df = pd.read_csv(class_URI.split("/")[-1]+"_query_results.tsv", sep = "\t")

In [72]:
predicate_counts = df['predicate'].value_counts()
print(predicate_counts)

predicate
http://www.w3.org/1999/02/22-rdf-syntax-ns#type    21554
http://purl.org/dc/terms/conformsTo                21554
https://schema.org/alternateName                   21554
https://schema.org/creator                         21554
https://schema.org/description                     21554
https://schema.org/distribution                    21554
https://schema.org/keywords                        21554
https://schema.org/name                            21554
https://schema.org/url                             21554
http://mlcommons.org/croissant/recordSet           20195
https://schema.org/license                          5065
https://schema.org/sameAs                           1883
https://schema.org/identifier                        106
Name: count, dtype: int64


In [73]:
predicate_counts.to_csv(class_URI.split("/")[-1]+"_counts.tsv", sep = "\t")